In [1]:
import pandas as pd
df = pd.read_csv('final_food_delivery_dataset.csv')
df.columns = df.columns.str.strip()
df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True)

# 1. City with highest total revenue from Gold members
gold_members = df[df['membership'].str.strip() == 'Gold']
city_rev = gold_members.groupby('city')['total_amount'].sum()
print(f"1. Highest Revenue City (Gold): {city_rev.idxmax()}")

# 2. Cuisine with highest average order value
cuisine_avg = df.groupby('cuisine')['total_amount'].mean()
print(f"2. Highest Avg Order Value Cuisine: {cuisine_avg.idxmax()}")

# 3. Distinct users with total orders > ₹1000
user_totals = df.groupby('user_id')['total_amount'].sum()
count_1000 = (user_totals > 1000).sum()
print(f"3. Users with total orders > ₹1000: {count_1000}")

# 4. Rating range with highest revenue
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0 – 3.5', '3.6 – 4.0', '4.1 – 4.5', '4.6 – 5.0']
df['rating_range'] = pd.cut(df['rating'], bins=bins, labels=labels, include_lowest=True)
rating_rev = df.groupby('rating_range')['total_amount'].sum()
print(f"4. Highest Revenue Rating Range: {rating_rev.idxmax()}")

# 5. Highest AOV for Gold members by city
gold_city_aov = gold_members.groupby('city')['total_amount'].mean()
print(f"5. Highest Gold AOV City: {gold_city_aov.idxmax()}")

# 6. Cuisine with lowest distinct restaurants but high revenue
cuisine_stats = df.groupby('cuisine').agg(
    unique_restaurants=('restaurant_id', 'nunique'),
    total_rev=('total_amount', 'sum')
)
print("\n6. Cuisine Comparison (Lowest count vs high revenue):")
print(cuisine_stats.sort_values(by='unique_restaurants'))

# 7. Percentage of total orders by Gold members
gold_pct = (len(gold_members) / len(df)) * 100
print(f"\n7. Percentage of Gold Orders: {round(gold_pct)}%")

# 8. Restaurant with highest AOV but < 20 orders
res_stats = df.groupby('restaurant_name_x').agg(
    avg_val=('total_amount', 'mean'), 
    order_count=('total_amount', 'count')
)
small_res = res_stats[res_stats['order_count'] < 20]
print(f"8. Highest AOV Restaurant (<20 orders): {small_res['avg_val'].idxmax()}")

# 9. Highest revenue combination (Membership + Cuisine)
combo_rev = df.groupby(['membership', 'cuisine'])['total_amount'].sum()
print(f"9. Top Revenue Combo: {combo_rev.idxmax()}")

# 10. Highest Revenue Quarter
df['quarter'] = df['order_date'].dt.to_period('Q')
quarter_rev = df.groupby('quarter')['total_amount'].sum()
print(f"10. Highest Revenue Quarter: {quarter_rev.idxmax()}")

1. Highest Revenue City (Gold): Chennai
2. Highest Avg Order Value Cuisine: Mexican
3. Users with total orders > ₹1000: 2544
4. Highest Revenue Rating Range: 4.6 – 5.0
5. Highest Gold AOV City: Chennai

6. Cuisine Comparison (Lowest count vs high revenue):
         unique_restaurants   total_rev
cuisine                                
Chinese                 120  1930504.65
Indian                  126  1971412.58
Italian                 126  2024203.80
Mexican                 128  2085503.09

7. Percentage of Gold Orders: 50%
8. Highest AOV Restaurant (<20 orders): Hotel Dhaba Multicuisine
9. Top Revenue Combo: ('Regular', 'Mexican')
10. Highest Revenue Quarter: 2023Q3


C:\Users\basva\AppData\Local\Temp\ipykernel_13744\2666210362.py:4: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True)
C:\Users\basva\AppData\Local\Temp\ipykernel_13744\2666210362.py:24: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rating_rev = df.groupby('rating_range')['total_amount'].sum()


In [2]:
import pandas as pd
df = pd.read_csv('final_food_delivery_dataset.csv')
df.columns = df.columns.str.strip()

# 1. Total orders placed by users with Gold membership
gold_orders = len(df[df['membership'] == 'Gold'])

# 2. Total revenue generated from Hyderabad (rounded to nearest integer)
hyd_rev = round(df[df['city'] == 'Hyderabad']['total_amount'].sum())

# 3. How many distinct users placed at least one order?
distinct_users = df['user_id'].nunique()

# 4. Average order value for Gold members (rounded to 2 decimals)
gold_aov = round(df[df['membership'] == 'Gold']['total_amount'].mean(), 2)

# 5. How many orders were placed for restaurants with rating >= 4.5?
high_rating_orders = len(df[df['rating'] >= 4.5])

# 6. How many orders were placed in the top revenue city among Gold members only?
gold_df = df[df['membership'] == 'Gold']
top_city = gold_df.groupby('city')['total_amount'].sum().idxmax()
top_city_gold_orders = len(gold_df[gold_df['city'] == top_city])

# Printing Results
print(f"1. Total Gold Orders: {gold_orders}")
print(f"2. Hyderabad Total Revenue: {hyd_rev}")
print(f"3. Distinct Users Count: {distinct_users}")
print(f"4. Gold Member AOV: {gold_aov}")
print(f"5. Rating >= 4.5 Orders: {high_rating_orders}")
print(f"6. Orders in Top Gold City ({top_city}): {top_city_gold_orders}")

1. Total Gold Orders: 4987
2. Hyderabad Total Revenue: 1889367
3. Distinct Users Count: 2883
4. Gold Member AOV: 797.15
5. Rating >= 4.5 Orders: 3374
6. Orders in Top Gold City (Chennai): 1337


In [3]:
import pandas as pd
import sqlite3
import json
orders = pd.read_csv('orders.csv')
with open('users.json', 'r') as f:
    users_data = json.load(f)
users = pd.DataFrame(users_data)
conn = sqlite3.connect(':memory:')
with open('restaurants.sql', 'r') as f:
    sql_script = f.read()
conn.executescript(sql_script)
restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)
merged_step1 = pd.merge(orders, users, on='user_id', how='left')
final_df = pd.merge(merged_step1, restaurants, on='restaurant_id', how='left')
final_df.columns = final_df.columns.str.strip()
final_df['order_date'] = pd.to_datetime(final_df['order_date'], dayfirst=True)
final_df.to_csv('final_food_delivery_dataset.csv', index=False)

#printing 3rd ans
print(f"Total Rows in Dataset: {len(final_df)}")

Total Rows in Dataset: 10000
